# 构造数据集

从wmo上爬取分类云图

In [ ]:
import os
from io import BytesIO

In [ ]:
import requests as rq
from bs4 import BeautifulSoup
from PIL import Image

In [ ]:
api_url = u"https://cloudatlas.wmo.int/api/?&tag=&cldg={cloud_type}&cldt=&sp=&met=&cz=&clcode=&cmcode=&chcode="
img_url = u"https://cloudatlas.wmo.int/images/compressed/{name}"

In [ ]:
def get_raw_response(cloud_type):
    url = api_url.format(cloud_type=cloud_type)
    res = rq.get(url)
    if 200 == res.status_code:
        return res.text
    else:
        return None

In [ ]:
keyword_list = [u"卷云",u"卷积云",u"卷层云",u"高积云",u"高层云",u"雨层云",u"层积云",u"层云",u"积云",u"积雨云"]
type_list = [u"H1",u"H2",u"H3",u"M1",u"M2",u"M3","L1",u"L2",u"L3",u"L4"]

In [ ]:
data_path = "data"
test_data_path = "test"

In [ ]:
#for kw in keyword_list:
#    os.system("mkdir %s"%os.path.join(test_data_path,kw))

In [ ]:
def parse_raw_response(res):
    names = []
    for line in res.strip().split('\n'):
        try:
            idx,_,name,_,author = line.strip().split(';')
            name,ext = os.path.splitext(name)
            if(name.endswith("_m")):
                name = name[:-2]
            full_name = "{idx}_main_{name}{ext}".format(idx=idx,name=name,ext=ext)
            names.append(full_name)
        except:
            print("err in {}".format(line))
        
    return names

In [ ]:
def download_img(img_name,path):
    url = img_url.format(name=img_name)
    full_path = os.path.join(path,img_name)
    print("downloading {}".format(url))
    if(os.path.exists(full_path)):
        return
    with open(full_path,"wb") as f:
        f.write(rq.get(url).content)

In [ ]:
for idx,path in enumerate(keyword_list):
    cloud_type = type_list[idx]
    data_save_path = os.path.join(data_path,path)
    test_save_path = os.path.join(test_data_path,path)
    print("handling {type}".format(type=path))
    raw = get_raw_response(cloud_type)
    names = parse_raw_response(raw)
    
    split_point = int(len(names)*0.8)
    data = names[:split_point]
    test = names[split_point:]
    
    for n in data:
        download_img(n,data_save_path)
    for n in test:
        download_img(n,test_save_path)

In [ ]:
raw